In [1]:
l1 = [1, 2, 3]
l2 = [4, 5, 6]

zip(l1, l2)

[(1, 4), (2, 5), (3, 6)]

In [6]:
import cPickle
import multiprocessing

In [4]:
name_prefix = 'thread-'
name_postfix = '.dump'

In [7]:
W_pieces = [cPickle.load(open(name, 'r')) for name in [name_prefix + str(en) + name_postfix for en, x in enumerate(range(0,9))]]
W = {}
map(lambda x: W.update(x), W_pieces)

[None, None, None, None, None, None, None, None, None]

In [8]:
len(W)

1200

In [11]:
try2_W = cPickle.load(open('try2.output.dmp', 'r'))

In [12]:
len(try2_W)

1200

In [13]:
set(W.keys()) == set(try2_W.keys())

True

In [14]:
W.keys()[0]

'4303'

In [15]:
try2_W.keys()[0]

'1142'

In [17]:
W['4303']

[('2226', 0.92601246),
 ('843', 0.91736633),
 ('3888', 0.91626364),
 ('3460', 0.91508913),
 ('989', 0.91380954),
 ('3277', 0.89689541),
 ('3380', 0.89161569),
 ('2619', 0.89090306),
 ('3748', 0.8904385),
 ('2703', 0.88784641),
 ('134', 0.88620925),
 ('658', 0.87769818),
 ('651', 0.87569541),
 ('859', 0.8728919),
 ('3482', 0.86906517),
 ('3517', 0.86695516),
 ('3413', 0.86299044),
 ('3065', 0.85653186),
 ('1886', 0.85407102),
 ('887', 0.85145432)]

In [18]:
try2_W['4303']

[('2226', 0.94935459),
 ('3888', 0.94291461),
 ('843', 0.93947029),
 ('3460', 0.92827022),
 ('989', 0.92502898),
 ('859', 0.92001593),
 ('2619', 0.91479921),
 ('134', 0.91462708),
 ('3380', 0.91417456),
 ('3277', 0.91013694),
 ('658', 0.90172535),
 ('2703', 0.90051883),
 ('651', 0.89631301),
 ('1886', 0.89338088),
 ('1714', 0.88087994),
 ('1386', 0.87979919),
 ('572', 0.87903476),
 ('3413', 0.87831783),
 ('3748', 0.87519908),
 ('3065', 0.87279391)]

In [19]:
import os
import gensim#from gensim.models import word2vec
import math
import random
import csv
import numpy as np
from numpy import linalg as la
import heapq
import multiprocessing
#from multiprocessing.dummy import Pool as ThreadPool
import datetime
import logging
#import threading
#import thread
import cPickle
import sqlite3
from utility_extract_data import extract_data_from_file_and_generate_train_and_test
from utility_user_repr import *

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


class RecommendatorSystem(object):
    """docstring for RecommendatorSystem"""
    def __init__(self):
        super(RecommendatorSystem, self).__init__()
        
    def setup(self):
        assert(False)

    def split_data(self, data, M, k, seed):
        '''Possible problem: 
        '''
        test = []
        train = []
        random.seed(seed)
        for user, item in data.items():
            if k == random.randint(0, M):
                test.append([user, item])
            else:
                train.append([user, item])
        return train, test


    def calculate_metrics(self, train, test, N):
        starttime = datetime.datetime.now()

        ###
        threads = []
        # Start consumers
        num_threads = multiprocessing.cpu_count() * 2
        #num_threads = 1 # for debugging        
        if len(test) < num_threads:
            num_threads = 1
        
        print 'Creating %d threads' % num_threads

        piece_len = len(test) / num_threads

        test__in_list = test.items()
        #print 'dict(test__in_list):', dict(test__in_list)
        pieces = [dict(test__in_list[x * piece_len: (x + 1) * piece_len]) for x in xrange(0, num_threads + 1)]
        #print 'pieces:', pieces
        # 创建线程对象
        name_prefix = 'thread-'
        name_postfix = '.dump'
        for en, x in enumerate(pieces):
            name = name_prefix + str(en) + name_postfix
            threads.append(InnerThreadClass(name, self, len(test), x, N))
        for t in threads:
            t.start()
        for t in threads:
            t.join()  

        total = len(test)
        print 'progress: %d/%d. done.' % (total, total)

        #
        rec_pieces = [cPickle.load(open(name, 'r')) for name in [name_prefix + str(en) + name_postfix for en in xrange(num_threads + 1)]]
        #print 'rec_pieces:', rec_pieces
        rec = {}
        map(lambda x: rec.update(x), rec_pieces)
        
        #print 'rec:', rec
        ###

        hit = 0
        all__for_recall = 0
        all__for_precision = 0
        for user in test.keys():
            history = test[user][0]
            answer = test[user][1]
            tu = [x[0] for x in answer]
            rank = rec[user] # self.recommend(history, N)
            #print 'rank:', rank
            for item, pui in rank:
                if item in tu:
                    hit += 1
            all__for_recall += len(tu)
            all__for_precision += len(rank) #Note: In book RSP, the author used 'all += N'
        
        metric_recall = None
        metric_precision = None
        metric_f1 = None
        if 0 == all__for_recall:
            metric_recall = 0
        else:
            metric_recall = hit / (all__for_recall * 1.0)
        
        if 0 == all__for_precision:
            metric_precision = 0
        else:
            metric_precision = hit / (all__for_precision * 1.0)

        if 0 == all__for_recall or 0 == all__for_precision:
            metric_f1 = 0
        else:
            metric_f1 = 2/(1./metric_precision + 1./metric_recall)

        endtime = datetime.datetime.now()
        interval = (endtime - starttime).seconds
        print 'metric calculation: time consumption: %d' % (interval)
        return {'recall': metric_recall, 'precision': metric_precision, 'f1': metric_f1}

    
### Word2vec based CB
class RecommendatorViaWord2VecBasedCB(RecommendatorSystem):
    def __init__(self):
        super(RecommendatorSystem, self).__init__()

    def setup(self, para):
        item_file_name, item_file_delimiter, genre_delimiter = para['item_file_name'], para['item_file_delimiter'], para['genre_delimiter']
        rating_file_name, rating_file_delimiter = para['rating_file_name'], para['rating_file_delimiter']
        model_path = para['model_path']
        self.user_histor2user_repr_function = para['user_histor2user_repr_function']

        self.item_info = self.extract_item_info(item_file_name, item_file_delimiter, genre_delimiter)
        self.user_item_interaction = extract_user_item_interaction(rating_file_name, rating_file_delimiter)
        self.model = gensim.models.Word2Vec.load(model_path)
        self.user_repr = {user: self.user_histor2user_repr_function(self.model, self.user_item_interaction[user]) 
                     for user in self.user_item_interaction}
        self.item_repr = self.model
        self.all_items = set(self.model.wv.vocab.keys())

        self.total_user_item_comb = 0


    def extract_item_info(self, filename, delimiter, genre_delimiter):
        data = {}

        with open(filename , 'r') as f:
            for i, line in enumerate(f):
                itemId, title, genre_list = map(lambda x: x.strip(), line.split(delimiter))

                data[itemId] = (title, genre_list.split(genre_delimiter))
        return data


    def extract_user_item_interaction(self, filename, delimiter):
        data = {}

        with open(filename , 'r') as f:
            for i, line in enumerate(f):
                userId, movieId, rating, timestamp = line.split(delimiter)
                #userId = int(userId)
                #movieId = int(movieId)
                rating = float(rating)
                timestamp = int(timestamp)

                if userId not in data:
                    data[userId] = []
                data[userId].append((movieId, rating, timestamp))

        # order by time
        for userId in data:
            data[userId].sort(key=lambda x: x[2]) 
        return data

        '''
    def main():
        item_file_name, item_file_delimiter, genre_delimiter = os.path.sep.join(['ml-1m', 'movies.dat']), '::', '|'
        item_info = extract_item_info(item_file_name, item_file_delimiter, genre_delimiter)


        rating_file_name, rating_file_delimiter = os.path.sep.join(['ml-1m', 'ratings.dat']), '::'
        user_item_interaction = extract_user_item_interaction(rating_file_name, rating_file_delimiter)

        model_path = '/home/wsyj/dissertation__recommendation_system__experiment_2/dissertation__recommendation_system__experiment/main_modelnum_features=100_min_count=1_window=1_iter=30.model'

        #model = gensim.models.Word2Vec.load('/home/wsyj/dissertation__recommendation_system__experiment_2/dissertation__recommendation_system__experiment/main_modelnum_features=200_min_count=5_window=2.model' )
        model = gensim.models.Word2Vec.load(model_path)

        #user_history2user_repr__simple_average(model, user_item_interaction['5989'])   

        # calculate user representation dict
        user_repr = {user: user_history2user_repr__simple_average(model, user_item_interaction[user]) 
                     for user in user_item_interaction}

        # item representation
        item_repr = model
        #print len(item_repr)

        all_items = set(model.wv.vocab.keys())

        # load train and test datasets

        data_filename, delimiter, data_set = os.path.sep.join(['ml-1m', 'ratings.dat']), '::', '1M'
        #data_filename, delimiter = os.path.sep.join(['ml-10M100K', 'ratings.dat']), '::'
        #data_filename, delimiter, data_set = os.path.sep.join(['ml-100k', 'u.data']), '\t', '100K'

        N = 20
        seed = 2 
        K = 10
        train_percent = 0.8
        test_data_inner_ratio = 0.8
        test = None
        train, original_test = extract_data_from_file_and_generate_train_and_test(data_filename, train_percent, seed, delimiter, test_data_inner_ratio)
        #train, test = extract_data_from_file_and_generate_train_and_test(data_filename, 3, 0, seed, delimiter)


        # main: core of content-based recommendation

        total_user_item_comb = 0
        rec = {}
        for user in original_test:
            history, future = original_test[user]
            history_items = set([x[0] for x in history])
            candidates = all_items - history_items # filtering out those interacted
            #print 'candidates:', candidates

            total_user_item_comb += len(candidates)
            cand_simi_list = []
            for candy in candidates:
                simi = user_repr[user].dot(item_repr[candy]) / (la.norm(user_repr[user]) * la.norm(item_repr[candy]))
                cand_simi_list.append((candy, simi))

            cand_simi_list.sort(key=lambda x: -1 * x[1])

            rec[user] = cand_simi_list[:N]

        print 'total_user_item_comb:', total_user_item_comb

        metrics = calculate_metrics(original_test, rec)
        print "metrics:", metrics
'''
    
    def recommend(self, target_user_history, N, K=10):
        '''@K: number of user neighbors considered
        '''
        print 'word2vec based CB'

        history_items = set([x[0] for x in target_user_history])
        target_user_repr = self.user_histor2user_repr_function(self.model, target_user_history)
        candidates = self.all_items - history_items # filtering out those interacted
        #print 'candidates:', candidates

        self.total_user_item_comb += len(candidates)
        cand_simi_list = []
        for candy in candidates:
            simi = target_user_repr.dot(self.item_repr[candy]) / (la.norm(target_user_repr) * la.norm(self.item_repr[candy]))
            cand_simi_list.append((candy, simi))

        cand_simi_list.sort(key=lambda x: -1 * x[1])

        return cand_simi_list[:N]
        #####################

'''
        rank = {}
        interacted_items = [x[0] for x in target_user_history]
        #print 'target_user_history:', target_user_history
        #print 'interacted_items:', interacted_items

        K_neighbors = self.find_K_neighbors(target_user_history, K)

        for v, wuv in K_neighbors:
        #for v, wuv in sorted(self.W[u].items(), key=lambda x: x[1], reverse=True)[0:K]: # wuv: similarity between user u and user v
            for i, rvi, timestamp in self.train[v]: # rvi: rate of item by user v
                if i in interacted_items:
                    #do not recommend items which user u interacted before
                    continue

                if i not in rank:
                    rank[i] = 0.0
                rank[i] += wuv * rvi

        rank = rank.items()
        rank.sort(key=lambda x: x[1], reverse=True)
        return rank[:N]'''


def print_matrix(M):
    def print_wrapper(x):
        print x, M[x]
    map(lambda x: print_wrapper(x), M)


In [34]:
rs = RecommendatorViaWord2VecBasedCB()
rs.setup({'item_file_name': os.path.sep.join(['ml-1m', 'movies.dat']), 
        'item_file_delimiter': '::',
        'genre_delimiter': '|',

        'rating_file_name': os.path.sep.join(['ml-1m', 'ratings.dat']),
        'rating_file_delimiter': '::',

        'model_path': '/home/wsyj/dissertation__recommendation_system__experiment_2/dissertation__recommendation_system__experiment/main_modelnum_features=100_min_count=1_window=1_iter=30.model',

        'user_histor2user_repr_function': user_history2user_repr__simple,
})

2017-04-06 16:24:49,532 : INFO : loading Word2Vec object from /home/wsyj/dissertation__recommendation_system__experiment_2/dissertation__recommendation_system__experiment/main_modelnum_features=100_min_count=1_window=1_iter=30.model
2017-04-06 16:24:49,542 : INFO : loading wv recursively from /home/wsyj/dissertation__recommendation_system__experiment_2/dissertation__recommendation_system__experiment/main_modelnum_features=100_min_count=1_window=1_iter=30.model.wv.* with mmap=None
2017-04-06 16:24:49,542 : INFO : setting ignored attribute syn0norm to None
2017-04-06 16:24:49,543 : INFO : setting ignored attribute cum_table to None
2017-04-06 16:24:49,543 : INFO : loaded /home/wsyj/dissertation__recommendation_system__experiment_2/dissertation__recommendation_system__experiment/main_modelnum_features=100_min_count=1_window=1_iter=30.model


In [51]:
data_filename, delimiter, data_set = os.path.sep.join(['ml-1m', 'ratings.dat']), '::', '1M'
#data_filename, delimiter = os.path.sep.join(['ml-10M100K', 'ratings.dat']), '::'
#data_filename, delimiter, data_set = os.path.sep.join(['ml-100k', 'u.data']), '\t', '100K'

seed = 2 
K = 10
train_percent = 0.8
test_data_inner_ratio = 0.8
train, test = extract_data_from_file_and_generate_train_and_test(data_filename, train_percent, seed, delimiter, test_data_inner_ratio)

sort_by_time: False


In [25]:
len(test)

1200

In [52]:
# def recommend(self, target_user_history, N, K=10):

target_user_id = '4303'
rs.recommend(test[target_user_id][0], 20)

word2vec based CB


[('2226', 0.92601246),
 ('843', 0.91736633),
 ('3888', 0.91626364),
 ('3460', 0.91508913),
 ('989', 0.91380954),
 ('3277', 0.89689541),
 ('3380', 0.89161569),
 ('2619', 0.89090306),
 ('3748', 0.8904385),
 ('2703', 0.88784641),
 ('134', 0.88620925),
 ('658', 0.87769818),
 ('651', 0.87569541),
 ('859', 0.8728919),
 ('3482', 0.86906517),
 ('3517', 0.86695516),
 ('3413', 0.86299044),
 ('3065', 0.85653186),
 ('1886', 0.85407102),
 ('887', 0.85145432)]

In [57]:
rs.item_repr['2226']

array([-0.07995249, -0.03568274,  0.10432107,  0.11091466, -0.01100429,
       -0.17950441,  0.10763099, -0.11357614,  0.16079967,  0.01536572,
        0.16512679,  0.03932391, -0.01480475, -0.09908679,  0.06570812,
       -0.11590304, -0.10808569, -0.05665402,  0.05365364,  0.0219524 ,
       -0.04337854,  0.15584807, -0.03881831,  0.08431357,  0.02600622,
       -0.12639721,  0.06112057, -0.1396686 ,  0.15059343,  0.09106894,
       -0.04577739, -0.05692285, -0.10817996, -0.12845665,  0.01696131,
       -0.05179565,  0.0818283 ,  0.15346935, -0.04456283,  0.11329725,
        0.09736568, -0.01164295, -0.04671535,  0.10337068,  0.09236275,
        0.07060608, -0.02350138, -0.16532132,  0.01778999, -0.10881312,
        0.04054732,  0.02967958,  0.13123903,  0.05488566,  0.13757376,
       -0.07669593,  0.20684645, -0.08336023, -0.10686736,  0.08607247,
        0.030428  , -0.01641421,  0.03212703, -0.22401127,  0.10563564,
        0.01048131, -0.12061931, -0.1811237 , -0.21279833, -0.02

In [61]:
rs.user_repr[target_user_id].dot(rs.item_repr['2226']) /   (la.norm(rs.user_repr[target_user_id]) * la.norm(rs.item_repr['2226']))


0.94935459

In [62]:
rs.user_histor2user_repr_function(rs.model, test[target_user_id][0])

array([ -3.14346179e-02,  -2.43079253e-02,   4.00689878e-02,
         3.87215279e-02,   1.89239606e-02,  -7.65962079e-02,
         2.18911432e-02,  -5.04102521e-02,   1.05822757e-01,
        -8.34232010e-03,   8.64890739e-02,   1.88324098e-02,
        -1.43516874e-02,  -2.19456460e-02,   3.88069265e-02,
        -8.38797987e-02,  -3.20274644e-02,  -5.34103960e-02,
         1.31921144e-02,   2.05511693e-02,  -2.97581889e-02,
         6.00219294e-02,  -1.49239525e-02,   2.18010638e-02,
         2.54599079e-02,  -4.32962291e-02,   4.06921357e-02,
        -1.11218400e-01,   5.54783754e-02,   2.80525591e-02,
        -1.89394429e-02,  -3.91659401e-02,  -2.73492374e-02,
        -6.32353276e-02,   3.01409122e-02,   3.50768282e-03,
         4.09195423e-02,   7.28367493e-02,  -4.99162935e-02,
         5.90917021e-02,   1.50144733e-02,  -3.69294174e-02,
         3.81200155e-03,   6.61503747e-02,   3.84372994e-02,
         1.30061200e-02,  -2.80600991e-02,  -9.07633752e-02,
        -3.69095244e-02,

In [59]:
item_repr['2226']
user_repr[target_user_id].dot(item_repr['2226']) / (la.norm(user_repr[target_user_id]) * la.norm(item_repr['2226']))

0.94935459

In [53]:
user = target_user_id
history, future = original_test[user]
history_items = set([x[0] for x in history])
candidates = all_items - history_items # filtering out those interacted
#print 'candidates:', candidates

total_user_item_comb += len(candidates)
cand_simi_list = []
for candy in candidates:
    simi = user_repr[user].dot(item_repr[candy]) / (la.norm(user_repr[user]) * la.norm(item_repr[candy]))
    cand_simi_list.append((candy, simi))

    cand_simi_list.sort(key=lambda x: -1 * x[1])

    

In [55]:
user_repr[user]

array([-0.02998144, -0.00725607,  0.0540598 ,  0.04305745,  0.00356386,
       -0.09771697,  0.01962578, -0.05186418,  0.10864354,  0.01012072,
        0.09448522,  0.01894226, -0.01534213, -0.02943856,  0.05182371,
       -0.07218101, -0.04372488, -0.03683557,  0.01982726,  0.00111637,
       -0.02494859,  0.06807517, -0.01570932,  0.01937005,  0.01860823,
       -0.06247325,  0.05167216, -0.09294701,  0.0587185 ,  0.03370877,
       -0.02212591, -0.03227014, -0.03925757, -0.06762307,  0.0434934 ,
       -0.00750151,  0.04712634,  0.08051383, -0.03738636,  0.07138214,
        0.00261551, -0.03158722, -0.01819723,  0.05965479,  0.06039528,
        0.02843246, -0.01186906, -0.08832191, -0.01572738, -0.04541136,
        0.01458372,  0.03037676,  0.06580335,  0.0065573 ,  0.06968792,
       -0.03493665,  0.09015711, -0.04899194, -0.06306034,  0.04478908,
        0.04816107,  0.00214435,  0.01574012, -0.10736565,  0.04368715,
        0.02015235, -0.04464936, -0.08446606, -0.0740968 , -0.02

In [54]:
cand_simi_list

[('2226', 0.94935459),
 ('3888', 0.94291461),
 ('843', 0.93947029),
 ('3460', 0.92827022),
 ('989', 0.92502898),
 ('859', 0.92001593),
 ('2619', 0.91479921),
 ('134', 0.91462708),
 ('3380', 0.91417456),
 ('3277', 0.91013694),
 ('658', 0.90172535),
 ('2703', 0.90051883),
 ('651', 0.89631301),
 ('1886', 0.89338088),
 ('1714', 0.88087994),
 ('1386', 0.87979919),
 ('572', 0.87903476),
 ('3413', 0.87831783),
 ('3748', 0.87519908),
 ('3065', 0.87279391),
 ('1315', 0.86692423),
 ('3517', 0.8665778),
 ('1905', 0.86530614),
 ('3291', 0.85867208),
 ('3482', 0.85815883),
 ('3315', 0.85647124),
 ('3656', 0.85557282),
 ('2909', 0.8531782),
 ('2223', 0.85290682),
 ('2214', 0.85197717),
 ('311', 0.85053724),
 ('398', 0.8473599),
 ('2218', 0.8470124),
 ('142', 0.8462041),
 ('130', 0.84597808),
 ('127', 0.84519923),
 ('672', 0.84346282),
 ('2811', 0.8411147),
 ('887', 0.84048045),
 ('1528', 0.84038728),
 ('3631', 0.83652323),
 ('878', 0.83640128),
 ('3229', 0.83629823),
 ('584', 0.83409935),
 ('2343', 

In [35]:
rs.item_info == item_info

True

In [36]:
rs.user_item_interaction == user_item_interaction

True

In [40]:
rs.user_repr.keys() == user_repr.keys()

True

In [41]:
rs.user_repr.values() == user_repr.values()

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [48]:
print rs.user_repr[target_user_id]
print user_repr[target_user_id]

[-0.02998144 -0.00725607  0.0540598   0.04305745  0.00356386 -0.09771697
  0.01962578 -0.05186418  0.10864354  0.01012072  0.09448522  0.01894226
 -0.01534213 -0.02943856  0.05182371 -0.07218101 -0.04372488 -0.03683557
  0.01982726  0.00111637 -0.02494859  0.06807517 -0.01570932  0.01937005
  0.01860823 -0.06247325  0.05167216 -0.09294701  0.0587185   0.03370877
 -0.02212591 -0.03227014 -0.03925757 -0.06762307  0.0434934  -0.00750151
  0.04712634  0.08051383 -0.03738636  0.07138214  0.00261551 -0.03158722
 -0.01819723  0.05965479  0.06039528  0.02843246 -0.01186906 -0.08832191
 -0.01572738 -0.04541136  0.01458372  0.03037676  0.06580335  0.0065573
  0.06968792 -0.03493665  0.09015711 -0.04899194 -0.06306034  0.04478908
  0.04816107  0.00214435  0.01574012 -0.10736565  0.04368715  0.02015235
 -0.04464936 -0.08446606 -0.0740968  -0.02223224  0.02084603 -0.00617117
 -0.01916723 -0.07439318  0.0144606  -0.02678853  0.0185461   0.00688943
  0.05380209  0.05691454 -0.10001567  0.03852278 -0.

In [45]:
rs.item_repr.vocab.keys()[:10]

2017-04-06 16:27:40,705 : WARNING : direct access to vocab will not be supported in future gensim releases, please use model.wv.vocab


['2031', '1869', '1868', '643', '344', '345', '346', '347', '340', '341']

In [46]:
rs.item_repr['2031']

array([-0.04877592,  0.01849231,  0.10109659,  0.1397278 ,  0.12087935,
       -0.04696242,  0.1468624 , -0.07833583,  0.02836458, -0.17617485,
        0.12662537,  0.08742664, -0.04179047,  0.0499889 ,  0.03761353,
       -0.13730173, -0.1029531 ,  0.05231122,  0.05665444, -0.02207289,
        0.01274252,  0.24177912,  0.03602125,  0.0461578 ,  0.13127564,
       -0.0843027 ,  0.03458912, -0.1926301 ,  0.08642396,  0.13020296,
       -0.12585181, -0.01364238,  0.08896869, -0.06179394,  0.02958413,
       -0.1513056 , -0.02232856,  0.08107794, -0.11408406,  0.13833445,
        0.02560141, -0.02423916, -0.11250423, -0.00788555,  0.18677661,
       -0.11209998,  0.13096553, -0.18044022,  0.11421954, -0.00474986,
       -0.00888   ,  0.07077996,  0.10984486,  0.02613745, -0.01384122,
        0.03172965,  0.10286706, -0.09545483, -0.17666063,  0.07804651,
        0.0917109 ,  0.04619632,  0.07485413, -0.09183019, -0.00828521,
       -0.06225332, -0.12762855, -0.19186321, -0.00336274, -0.04

In [47]:
item_repr['2031']

array([-0.04877592,  0.01849231,  0.10109659,  0.1397278 ,  0.12087935,
       -0.04696242,  0.1468624 , -0.07833583,  0.02836458, -0.17617485,
        0.12662537,  0.08742664, -0.04179047,  0.0499889 ,  0.03761353,
       -0.13730173, -0.1029531 ,  0.05231122,  0.05665444, -0.02207289,
        0.01274252,  0.24177912,  0.03602125,  0.0461578 ,  0.13127564,
       -0.0843027 ,  0.03458912, -0.1926301 ,  0.08642396,  0.13020296,
       -0.12585181, -0.01364238,  0.08896869, -0.06179394,  0.02958413,
       -0.1513056 , -0.02232856,  0.08107794, -0.11408406,  0.13833445,
        0.02560141, -0.02423916, -0.11250423, -0.00788555,  0.18677661,
       -0.11209998,  0.13096553, -0.18044022,  0.11421954, -0.00474986,
       -0.00888   ,  0.07077996,  0.10984486,  0.02613745, -0.01384122,
        0.03172965,  0.10286706, -0.09545483, -0.17666063,  0.07804651,
        0.0917109 ,  0.04619632,  0.07485413, -0.09183019, -0.00828521,
       -0.06225332, -0.12762855, -0.19186321, -0.00336274, -0.04

In [38]:
rs.item_repr == item_repr

False

In [39]:
rs.all_items == all_items

True

In [33]:

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import gensim
from gensim import corpora, models, similarities
import os
import utility_synopsis
import math
import numpy as np
from numpy import linalg as la
from scipy.spatial.distance import cosine
import pandas as pd
import matplotlib.pyplot as plt
from utility_extract_data import extract_data_from_file_and_generate_train_and_test
import datetime
import cPickle

def extract_item_info(filename, delimiter, genre_delimiter):
    data = {}

    with open(filename , 'r') as f:
        for i, line in enumerate(f):
            itemId, title, genre_list = map(lambda x: x.strip(), line.split(delimiter))
            
            data[itemId] = (title, genre_list.split(genre_delimiter))
    return data


def extract_user_item_interaction(filename, delimiter):
    data = {}

    with open(filename , 'r') as f:
        for i, line in enumerate(f):
            userId, movieId, rating, timestamp = line.split(delimiter)
            #userId = int(userId)
            #movieId = int(movieId)
            rating = float(rating)
            timestamp = int(timestamp)

            if userId not in data:
                data[userId] = []
            data[userId].append((movieId, rating, timestamp))
    
    # order by time
    for userId in data:
        data[userId].sort(key=lambda x: x[2]) 
    return data


def user_history2user_repr__simple_average(model, target_user_history): # target_user_history: It should_be_a_list_of_tuples_included_items.
    #print 'target_user_history:', target_user_history
    items_existed_in_model = filter(lambda x: x[0] in model, target_user_history)
    #print 'items_existed_in_model:', items_existed_in_model[0]
    items_translated_to_vecs = map(lambda x: model[x[0]], items_existed_in_model)
    #print 'items_translated_to_vecs:', items_translated_to_vecs[0]
    return np.average(items_translated_to_vecs, axis=0)   


def calculate_metrics(test, rec):
    starttime = datetime.datetime.now()
    hit = 0

    all__for_recall = 0
    all__for_precision = 0
    for user in test.keys():
        history = test[user][0]
        answer = test[user][1]
        tu = [x[0] for x in answer]
        rank = rec[user] # self.recommend(history, N)
        #print 'rank:', rank
        for item, pui in rank:
            if item in tu:
                hit += 1
        all__for_recall += len(tu)
        all__for_precision += len(rank) #Note: In book RSP, the author used 'all += N'

    metric_recall = None
    metric_precision = None
    metric_f1 = None
    if 0 == all__for_recall:
        metric_recall = 0
    else:
        metric_recall = hit / (all__for_recall * 1.0)

    if 0 == all__for_precision:
        metric_precision = 0
    else:
        metric_precision = hit / (all__for_precision * 1.0)

    if 0 == all__for_recall or 0 == all__for_precision:
        metric_f1 = 0
    else:
        metric_f1 = 2/(1./metric_precision + 1./metric_recall)

    endtime = datetime.datetime.now()
    interval = (endtime - starttime).seconds
    print 'metric calculation: time consumption: %d' % (interval)
    return {'recall': metric_recall, 'precision': metric_precision, 'f1': metric_f1}



item_file_name, item_file_delimiter, genre_delimiter = os.path.sep.join(['ml-1m', 'movies.dat']), '::', '|'
item_info = extract_item_info(item_file_name, item_file_delimiter, genre_delimiter)


rating_file_name, rating_file_delimiter = os.path.sep.join(['ml-1m', 'ratings.dat']), '::'
user_item_interaction = extract_user_item_interaction(rating_file_name, rating_file_delimiter)

model_path = '/home/wsyj/dissertation__recommendation_system__experiment_2/dissertation__recommendation_system__experiment/main_modelnum_features=100_min_count=1_window=1_iter=30.model'

#model = gensim.models.Word2Vec.load('/home/wsyj/dissertation__recommendation_system__experiment_2/dissertation__recommendation_system__experiment/main_modelnum_features=200_min_count=5_window=2.model' )
model = gensim.models.Word2Vec.load(model_path)

#user_history2user_repr__simple_average(model, user_item_interaction['5989'])   

# calculate user representation dict
user_repr = {user: user_history2user_repr__simple_average(model, user_item_interaction[user]) 
             for user in user_item_interaction}

# item representation
item_repr = model
#print len(item_repr)

all_items = set(model.wv.vocab.keys())

# load train and test datasets

data_filename, delimiter, data_set = os.path.sep.join(['ml-1m', 'ratings.dat']), '::', '1M'
#data_filename, delimiter = os.path.sep.join(['ml-10M100K', 'ratings.dat']), '::'
#data_filename, delimiter, data_set = os.path.sep.join(['ml-100k', 'u.data']), '\t', '100K'

N = 20
seed = 2 
K = 10
train_percent = 0.8
test_data_inner_ratio = 0.8
test = None
train, original_test = extract_data_from_file_and_generate_train_and_test(data_filename, train_percent, seed, delimiter, test_data_inner_ratio)
#train, test = extract_data_from_file_and_generate_train_and_test(data_filename, 3, 0, seed, delimiter)


# main: core of content-based recommendation

total_user_item_comb = 0
rec = {}
for user in original_test:
    history, future = original_test[user]
    history_items = set([x[0] for x in history])
    candidates = all_items - history_items # filtering out those interacted
    #print 'candidates:', candidates

    total_user_item_comb += len(candidates)
    cand_simi_list = []
    for candy in candidates:
        simi = user_repr[user].dot(item_repr[candy]) / (la.norm(user_repr[user]) * la.norm(item_repr[candy]))
        cand_simi_list.append((candy, simi))

    cand_simi_list.sort(key=lambda x: -1 * x[1])

    rec[user] = cand_simi_list[:N]

print 'total_user_item_comb:', total_user_item_comb

metrics = calculate_metrics(original_test, rec)
cPickle.dump(rec, open('try2.output.dmp', 'w'))
print "metrics:", metrics




2017-04-06 16:23:56,003 : INFO : loading Word2Vec object from /home/wsyj/dissertation__recommendation_system__experiment_2/dissertation__recommendation_system__experiment/main_modelnum_features=100_min_count=1_window=1_iter=30.model
2017-04-06 16:23:56,012 : INFO : loading wv recursively from /home/wsyj/dissertation__recommendation_system__experiment_2/dissertation__recommendation_system__experiment/main_modelnum_features=100_min_count=1_window=1_iter=30.model.wv.* with mmap=None
2017-04-06 16:23:56,012 : INFO : setting ignored attribute syn0norm to None
2017-04-06 16:23:56,013 : INFO : setting ignored attribute cum_table to None
2017-04-06 16:23:56,013 : INFO : loaded /home/wsyj/dissertation__recommendation_system__experiment_2/dissertation__recommendation_system__experiment/main_modelnum_features=100_min_count=1_window=1_iter=30.model


sort_by_time: False
total_user_item_comb: 4387464
metric calculation: time consumption: 0
metrics: {'recall': 6.8827000206481e-05, 'precision': 0.0004583333333333333, 'f1': 0.00011968164681946024}
